# Application

In [ ]:
 new_prod_recommendation(---------)

In [ ]:
# random loyalty card number _ 隨機選取有用的會員卡號
ran_member = random.choice(member_declare_sport_result_df['loyalty_card_num'])
print("The random loyalty card number is :" +" "+ran_member)
new_prod_recommendation(ran_member)

# Process

In [1]:
import psycopg2
import sys
import os
import time

In [3]:
import pandas as pd 
from decimal import *
import datetime
import random

In [4]:
def execute_query(query):
    try:
        conn = psycopg2.connect(dbname=DatabaseName, host=ClusterEndpoint, port=Port, user=UserName, password=Password)
        cur = conn.cursor()
        start = time.time()
        cur.execute(query)
        end = time.time()
        print('Time Taken in Seconds : '+str(round(end - start)))
        records = cur.fetchall()
        cur.close()
        conn.commit()
    except Exception as e:
       s = "UNEXPECTED ERROR: " + str(sys.exc_info())+"\n"+str(e)+"\nError on line {}".format(sys.exc_info()[-1].tb_lineno)
       print(s)
    finally:  
       conn.close()
    return records

# Declare Sport

In [ ]:
member_declare_sport = '''
SELECT t1.loyalty_card_num,
       t1.sport_id,
       t1.sport_name_zh,
       t1.sport_name_en,
       t2.classificaiton_code
FROM dtm_tw.tw_declare_sport AS t1
left join dtm_tw.taiwan_cdp_sport_tag AS t2
on t1.sport_id = t2.sport_id
where t1.sport_id != 'None'
group by 1,2,3,4,5
;
 '''

member_declare_sport_result = execute_query(member_declare_sport)
member_declare_sport_result_df = pd.DataFrame(member_declare_sport_result)
member_declare_sport_result_df.columns=['loyalty_card_num','sport_id','sport_name_zh','sport_name_en','classificaiton_code']

In [7]:
def member_declare_random_sport(member_id):
    spid = (member_declare_sport_result_df[member_declare_sport_result_df['loyalty_card_num'] == str(member_id)])
    one_random_sport = list(set(spid['sport_id']))
    one_random_sport_result = random.choice(one_random_sport)
    return one_random_sport_result

# Use the created  table for sport_id & model_code_r3 & p_sell_start_week

In [ ]:
sport_id_model_r3 = '''
SELECT distinct
    model_code_r3,  
    brand_id, 
    brand_name,
    p_sell_start_week, 
    web_label,      
    sports_id 
FROM dtm_tw.taiwan_cdp_sport_id_model_r3
WHERE p_sell_start_week > 202100
 ;
 ''' 

sport_id_model_r3_result = execute_query(sport_id_model_r3)
sport_id_model_r3_result_df = pd.DataFrame(sport_id_model_r3_result)
sport_id_model_r3_result_df.columns=['model_code_r3','brand_id','brand_nam','p_sell_start_week','web_label','sports_id']

# Capture the Week Num

In [10]:
from datetime import datetime
import calendar

In [11]:
def week_num():
    cal = datetime.today().isocalendar() 
    strcal = str(cal[0])+str(cal[1]) 
    return strcal

# Select Newest Product by sport_id & week_num_this_week

In [14]:
def recom_prod_new_2(member_id):
    one_random_sport_result = member_declare_random_sport(member_id)
    strcal = week_num()
    recom_prod_new = sport_id_model_r3_result_df.loc[(sport_id_model_r3_result_df['sports_id'] == one_random_sport_result) & (sport_id_model_r3_result_df['p_sell_start_week'] <= strcal)]
    if recom_prod_new.empty:
        return recom_prod_new
    else:
        uu = max(recom_prod_new['p_sell_start_week'])
        recom_prod_new_2 = sport_id_model_r3_result_df.loc[(sport_id_model_r3_result_df['sports_id'] == one_random_sport_result) & (sport_id_model_r3_result_df['p_sell_start_week'] == uu)]
        return recom_prod_new_2


 # Random Recommend 

In [15]:
def new_prod_recommendation(member_id):
    recom_prod_new_3 = recom_prod_new_2(member_id)
    if recom_prod_new_3.empty:
        print('No New Products')
    else:
        recom_prod_new_3 = recom_prod_new_2(member_id)
        if len(recom_prod_new_3) != 1 :
            a = random.randint(0,len(recom_prod_new_3))
            recom_prod_new_3.index = range(len(recom_prod_new_3))
            yy = recom_prod_new_3.iloc[a:a+3]
            return yy
        elif len(recom_prod_new_3) == 1:
            a = random.randint(0,len(recom_prod_new_3))
            recom_prod_new_3.index = range(len(recom_prod_new_3))
            kk = recom_prod_new_3.iloc[0:1]
            return kk
        elif len(recom_prod_new_3) == 0:
            a = random.randint(0,len(recom_prod_new_3))
            recom_prod_new_3.index = range(len(recom_prod_new_3))
            gg = recom_prod_new_3.iloc[0:0]
            return gg